In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from components.ovc_actif import actif_in_Q3Q4, actif_in_Q3strict, actif_in_Q4strict, actif_in_Q3, actif_in_Q4
from pandas import concat

from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [2]:
actif_in_Q3.id_patient.count()
actif_in_Q4.id_patient.count()

if not ((actif_in_Q3.id_patient.count() == actif_in_Q3Q4.id_patient.count() + actif_in_Q3strict.id_patient.count()) and (actif_in_Q4.id_patient.count() == actif_in_Q3Q4.id_patient.count() + actif_in_Q4strict.id_patient.count())):
    raise ValueError("Qi <> Qistrict + QiQj, bad logic")
else:
    print("the logic  Qi = Qistrict + QiQj is valid")

10888

14386

the logic  Qi = Qistrict + QiQj is valid


In [3]:
# actif_in_Q3Q4 = actif_in_Q3[actif_in_Q3.id_patient.isin(actif_in_Q4.id_patient)]

#### check the symetrie

actif_in_Q4Q3 = actif_in_Q4[actif_in_Q4.id_patient.isin(actif_in_Q3.id_patient)]
actif_in_Q4Q3.id_patient.count()

6861

In [4]:
actif_in_Q4strict.id_patient.count()
actif_in_Q3strict.id_patient.count()
actif_in_Q3Q4.id_patient.count()

7525

4027

6861

In [5]:
completed_Q4strict =  actif_in_Q4strict[actif_in_Q4strict.complete_at_least=="yes"]
completed_Q3strict =  actif_in_Q3strict[actif_in_Q3strict.complete_at_least=="yes"]
completed_Q3Q4 =  actif_in_Q3Q4[actif_in_Q3Q4.complete_at_least=="yes"]

completed_Q4strict.id_patient.count()
completed_Q3strict.id_patient.count()
completed_Q3Q4.id_patient.count()

7299

3866

6733

# Enrolled Q4

In [6]:
enrolledQ4_CQ4strict = completed_Q4strict[completed_Q4strict.isEnrolledQ4=="yes"]
#enrolledQ4_CQ3strict = completed_Q3strict[completed_Q3strict.isEnrolledQ4=="yes"]
enrolledQ4_CQ3Q4 = completed_Q3Q4[completed_Q3Q4.isEnrolledQ4=="yes"]


enrolledQ4_CQ4strict.id_patient.count()
#enrolledQ4_CQ3strict.id_patient.count()
#enrolledQ4_CQ3Q4.id_patient.count()




enrolledQ4_CQ3Q4[enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ4strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ4strict[enrolledQ4_CQ4strict.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()


#enrolledQ4_CQ4strict[~enrolledQ4_CQ4strict.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[~enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[~enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ4strict.id_patient)].id_patient.count()

1293

0

# Concatenation

In [7]:
all_ovc = concat([completed_Q3Q4,enrolledQ4_CQ4strict])

In [8]:
all_ovc.drop_duplicates("id_patient",inplace=True)
all_ovc.id_patient.count()
all_ovc.test_results.fillna("0,",inplace=True)
all_ovc.test_results.value_counts()

8026

0,          5583
0,,2,       1426
2,           999
1,             6
0,,3,          4
0,,1,,2,       2
0,,1,          2
3,             2
0,,2,,3,       1
2,,3,          1
Name: test_results, dtype: int64

In [9]:
ovc_data = all_ovc[
    (all_ovc.test_results!="0,,3,")&
    (all_ovc.test_results!="0,,1,,2,")&
    (all_ovc.test_results!="0,,2,,3,")&
    (all_ovc.test_results!="3,")&
    (all_ovc.test_results!="2,,3,")
]
ovc_data.id_patient.count()
ovc_data.test_results.value_counts()

8016

0,       5583
0,,2,    1426
2,        999
1,          6
0,,1,       2
Name: test_results, dtype: int64

# postive OVC_DREAMS

In [10]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()

8

In [11]:
positive_dreams

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,number_of_condoms_sensibilize,number_condoms_reception_in_the_interval,number_test_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4
564,85541,None,Ouest,Delmas,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
747,80897,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,secondary,no,no,complete,yes,no
1525,114955,None,Artibonite,Verrettes,8,0,yes,2,1,0,...,no,no,no,no,no,no,no,no,yes,no
2501,119259,None,Artibonite,Grande Saline,0,0,no,2,0,0,...,no,no,no,no,no,no,no,no,yes,no
2833,100506,None,Ouest,Tabarre,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
3469,109625,None,Artibonite,Petite Rivière de l'Artibonite,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
4852,125012,None,Ouest,Port-au-Prince,0,0,no,1,0,0,...,no,no,secondary,no,no,complete,no,no,yes,yes
4785,125650,NaN,Ouest,Port-au-Prince,21,0,yes,0,0,1,...,no,primary,no,no,secondary,no,no,no,yes,yes


# OVC_SERV_DREAMS

In [12]:
ovc_data.ovc_age.unique()

array(['18-24', '10-14', '15-17', '25-29', 'not_valid_age'], dtype=object)

In [13]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams.id_patient.count()
ovcS_dreams['Gender'] = "Female"

4224

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54937/4236012469.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


In [14]:
ovcS_dreams.parenting_detailed.unique()

array(['no'], dtype=object)

# Format Datim

In [15]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)

In [16]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams

Gender                         Female                        
ovc_age                            <1 1-4 5-9 10-14 15-17 18+
commune                                                      
Cap-Haïtien                         0   0   0   218   213   0
Carrefour                           0   0   0   137   114   0
Delmas                              0   0   0    82    50   0
Desdunes                            0   0   0    91    74   0
Dessalines                          0   0   0   193    87   0
Grande Saline                       0   0   0   112   118   0
Gressier                            0   0   0    77    41   0
Kenscoff                            0   0   0   392   333   0
La Chapelle                         0   0   0    30    32   0
Liancourt                           0   0   0    51    63   0
Limonade                            0   0   0    93    61   0
Milot                               0   0   0    56    29   0
Montrouis                           0   0   0    41    36   0
Petite Rivière de l'Artibonite      0   0   0    97   135   0
Plaine-du-Nord                      0   0   0    37    18   0
Port-au-Prince                      0   0   0    90   100   0
Pétionville                         0   0   0    66    72   0
Quartier-Morin                      0   0   0    33    23   0
Saint-Marc                          0   0   0   247   257   0
Tabarre                             0   0   0    30    24   0
Verrettes                           0   0   0    79    92   0

In [17]:
# datim_ovc_dreams.reset_index()

# Save the data

In [18]:
positive_dreams.to_excel('./ovc_results/positive_data/positive_dreams.xlsx',na_rep="",index=False)
datim_ovc_dreams.to_excel('./ovc_results/datim_ovc_dreams.xlsx',na_rep="",index=True)


#positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
#datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)

